### Step 1. Load required packages

In [1]:
import pandas as pd
import pandasql as ps
from pathlib import Path
workFolder = '/content/drive/Shareddrives/Esperanto/Supports/Suppliers/Work/1_Suppliers_Interface'

### Step 2. Read required data

In [2]:
reducedDf = pd.read_csv(Path(workFolder,'reducedDf.csv'), encoding='utf-8')
duplicateRecordsWithClassificationDf = pd.read_csv(Path(workFolder,'duplicateRecordsWithClassificationDf.csv'), encoding='utf-8')

### Step 3 - Identify Scenarios - Cases with more observations

In [3]:
sql_query = '''
SELECT numObs,COUNT(*) AS numCases FROM duplicateRecordsWithClassificationDf
GROUP BY numObs
ORDER BY numCases DESC
'''
result = ps.sqldf(sql_query)
for index, row in result.iterrows():
    print(row['numObs'],row['numCases'])

2 65407
3 11252
4 1634
5 25
6 6
7 1
8 1
10 1


### Step 4 - Analisis of specific scenario

In [4]:
sql_query = '''
SELECT multiple_feeds,multiples_ids,multiples_names,COUNT(*) AS numCases FROM
(SELECT * FROM duplicateRecordsWithClassificationDf
WHERE numObs=2)
GROUP BY multiple_feeds,multiples_ids,multiples_names
ORDER BY numCases DESC
'''
result = ps.sqldf(sql_query)
for index, row in result.iterrows():
    print(row['multiple_feeds'], row['multiples_ids'], row['multiples_names'], row['numCases'])

1 0 0 64670
1 0 1 733
1 1 0 3
0 1 0 1


In [5]:
print(reducedDf.head(100))

    Unnamed: 0 SISTEMA_LEGADO NUMERO_PROVEEDOR  \
0            0     CONSTRUNET            15239   
1            1     CONSTRUNET            15795   
2            2     CONSTRUNET            16109   
3            3     CONSTRUNET            18077   
4            4     CONSTRUNET            18785   
..         ...            ...              ...   
95          95         GASTOS    AIVC791215IM5   
96          96         GASTOS     AMG960314812   
97          97         GASTOS     ANO840306PL5   
98          98         GASTOS    AOCJ920325N61   
99          99         GASTOS    AOCP8411225Z1   

                               VENDOR_NAME       NUM_1099  
0             GRUPO SANDSTORM GAM SA DE CV   GSG010404212  
1          JORGE GUADALUPE RODRIGUEZ OCHOA  ROOJ570206L7A  
2                       SERNA RAMIREZ CRUZ  SERC660826UZ9  
3         ELIZABETH ARACELI RIVAS ESPINOZA  RIEE641221PS4  
4            CEMENTO EXPRESS S DE RL DE CV   CEX010613652  
..                                     

In [17]:
sql_query = '''
SELECT B.SISTEMA_LEGADO,B.NUMERO_PROVEEDOR,B.VENDOR_NAME,A.NUM_1099,
A.numObs,A.caseNum,A.multiple_feeds,A.multiples_ids,A.multiples_names
FROM
(SELECT NUM_1099,numObs,caseNum,multiple_feeds,multiples_ids,multiples_names
FROM duplicateRecordsWithClassificationDf
WHERE numObs=2 AND multiple_feeds=1 AND multiples_ids=0 AND multiples_names=0) AS A
LEFT JOIN
(SELECT SISTEMA_LEGADO,NUMERO_PROVEEDOR,VENDOR_NAME,NUM_1099 FROM reducedDf) AS B
ON (A.NUM_1099=B.NUM_1099)
ORDER BY A.NUM_1099
'''
result = ps.sqldf(sql_query)
print(result.head(10))
result.to_csv(str(Path(workFolder,'duplicateRecords_2Obs_multipleFeeds.csv', encoding='utf-8')))

  SISTEMA_LEGADO NUMERO_PROVEEDOR                   VENDOR_NAME  \
0         GASTOS     &JE040614N51  A & J EXPORTACIONES SA DE CV   
1          OBRAS     &JE040614N51  A & J EXPORTACIONES SA DE CV   
2         GASTOS    00AL570409DC8      OSORIO ALVAREZ JOSE LUIS   
3          OBRAS    00AL570409DC8      OSORIO ALVAREZ JOSE LUIS   
4         GASTOS    00BR5103152L7        ORDOÑEZ BARBA RAYMUNDO   
5          OBRAS    00BR5103152L7        ORDOÑEZ BARBA RAYMUNDO   
6         GASTOS    00CE810603TI4             OROSCO CORIA EMMA   
7          OBRAS    00CE810603TI4             OROSCO CORIA EMMA   
8         GASTOS     A&G060523IW9          DE ANDA & GRACIA S C   
9          OBRAS     A&G060523IW9          DE ANDA & GRACIA S C   

        NUM_1099  numObs  caseNum  multiple_feeds  multiples_ids  \
0   &JE040614N51       2    12921               1              0   
1   &JE040614N51       2    12921               1              0   
2  00AL570409DC8       2    12922               1         